Notebook to explore some approaches for determining and scoring how hierarchical a directed graph is.

**Definitions**:

- Hierarchy degree: This a network-wide metric that determines how "hierarchical" a graph is.
From https://doi.org/10.1186/s13059-015-0624-2:
    The degree of hierarchy for a given for a given network is not well-defined concept. Ispolatov et al. [27] introduced the idea of dominant direction by minimizing the number of feedback links. While it is a proxy of hierarchical structure to a certain extent, the method does not provide a rigorous statistical confidence. Here, we define a metric to quantify the degree of hierarchy for a given hierarchical network, and then propose a new method called hierarchical score maximization (HSM) to infer the hierarchy of a directed network

- Hierarchy metric (at node level): From sna package in R.
    Hierarchy measures quantify the extent of asymmetry in a structure; the greater the extent of asym-
    metry, the more hierarchical the structure is said to be. (This should not be confused with how
    centralized the structure is, i.e., the extent to which centralities of vertex positions are highly con-
    centrated.)
They discuss and offer 2 different approaches:

    - reciprocity: This setting returns one minus the dyadic reciprocity for each input graph (see
    grecip)
    - krackhardt: This setting returns the Krackhardt hierarchy score for each input graph. The
    Krackhardt hierarchy is defined as the fraction of non-null dyads in the reachability graph
    which are asymmetric. Thus, when no directed paths are reciprocated (e.g., in an in/outtree),
    Krackhardt hierarchy is equal to 1; when all such paths are reciprocated, by contrast (e.g., in
    a cycle or clique), the measure falls to 0.
        - Hierarchy is one of four measures (connectedness, efficiency, hierarchy, and lubness)
        suggested by Krackhardt for summarizing hierarchical structures. Each corresponds to one of
        four axioms which are necessary and sufficient for the structure in question to be an outtree;
        thus, the measures will be equal to 1 for a given graph iff that graph is an outtree. Deviations
        from unity can be interpreted in terms of failure to satisfy one or more of the outtree conditions,
        information which may be useful in classifying its structural properties.
        Note that hierarchy is inherently density-constrained: as densities climb above 0.5, the proportion
        of mutual dyads must (by the pigeonhole principle) increase rapidly, thereby reducing possibili-
        ties for asymmetry. Thus, the interpretation of hierarchy scores should take density into account,
        particularly if density is artifactual (e.g., due to a particular dichotomization procedure)

We will start by computing the Hierarchy degree as defined on its simplest form in the Cheng et al. 2015 paper. Then, we will try to use the R implementation through rpy2 and potentially develop our own.



In [12]:
import os, pandas as pd, numpy as np
import networkx

import rpy2, rpy2.situation
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr


In [2]:
for i in rpy2.situation.iter_info(): # Print Rpy2 info
    print(i)

pandas2ri.activate()
%load_ext rpy2.ipython

rpy2 version:
3.5.8
Python version:
3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:42:20) 
[Clang 14.0.6 ]
Looking for R's HOME:
    Environment variable R_HOME: /Users/ferran/miniconda3/envs/collabLite/lib/R
    Calling `R RHOME`: /Users/ferran/miniconda3/envs/collabLite/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/local/lib/R/library/stats/libs/:/usr/local/lib/R/library/stats/libs/
R version:
    In the PATH: R version 4.2.2 (2022-10-31) -- "Innocent and Trusting"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/Users/ferran/miniconda3/envs/collabLite/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'dl', 'm', 'iconv', 'icuuc', 'icui18n']
  library_dirs:
  ['/Users/ferran/miniconda3/envs/collabLite/lib', '/Users/ferran/miniconda3/envs/collabLite/lib/R/lib', '/Users/ferran/miniconda3/envs/collabLite/lib']
  extra_compile_args:
  ['-s

In [3]:
print(os.getcwd())
data_dir = f"{os.getcwd()}/data"

/Users/ferran/PhD/yaleCollab


In [4]:
dLRpath = pd.read_csv(f"{data_dir}/dLRpath.csv.tar.gz")
dRTpath = pd.read_csv(f"{data_dir}/dRTpath.csv.tar.gz")



dKGlrt = pd.read_csv(f"{data_dir}/dKGlrt.csv.tar.gz", header=None)
dKGlrt

,0,1,2
0,CXCL1,Signaling by GPCR,CXCR2
1,CXCL2,Signaling by GPCR,CXCR2
2,CXCL3,Signaling by GPCR,CXCR2
3,CXCL5,Signaling by GPCR,CXCR2
4,PPBP,Signaling by GPCR,CXCR2
...,...,...,...
97241,LGR4,Signaling by WNT,HIST1H2BO
97242,LGR4,Signaling by WNT,HIST1H2BH
97243,LGR4,Signaling by WNT,PSMB3
97244,LGR4,Signaling by WNT,HIST1H2BB


# HS 
Simple metric to determine the hierarchy degree of a network. From Cheng et al. 2015

Step 1: Assign levels to nodes

Step 2: Identify edge types

- Downward
- Upward
- Horizontal

HS = (Nd + Nh)/(Nu + Nh)

This metric will be affected by the number of levels present.
The resulting HS score goes from 0 to +inf, with fully hierarchical networks resulting in an infinite value


NOTE: THIS APPROACH HAS BEEN PAUSED AS WE HAVE NO UPWARD EDGES BY DEFINITION (due to the filtering of targets present either as ligands or receptors)
In the future, if we remove this filtering, we should explore this

Add level information as new column to dKGlrt:

In [9]:
dKGlrt[0].isin(dLRpath["head"]) & dKGlrt[2].isin(dLRpath["tail"])

0         True
1         True
2         True
3         True
4         True
         ...  
97241    False
97242    False
97243    False
97244    False
97245    False
Length: 97246, dtype: bool

In [10]:
# Nd = len(dKGlrt.loc[])

dKGlrt.drop(columns=[1]).loc[(dKGlrt[0].isin(dLRpath["head"]) & dKGlrt[2].isin(dLRpath["tail"])) | (dKGlrt[0].isin(dRTpath["head"]) & dKGlrt[2].isin(dRTpath["tail"])) | (dKGlrt[0].isin(dLRpath["head"]) & dKGlrt[2].isin(dLRpath["tail"]))].drop_duplicates()

,0,2
0,CXCL1,CXCR2
1,CXCL2,CXCR2
2,CXCL3,CXCR2
3,CXCL5,CXCR2
4,PPBP,CXCR2
...,...,...
97241,LGR4,HIST1H2BO
97242,LGR4,HIST1H2BH
97243,LGR4,PSMB3
97244,LGR4,HIST1H2BB


# Krackhardt hierarchy


In [22]:
G = networkx.from_pandas_edgelist(
    dKGlrt,
    source=0,target=2,
    edge_key=1,
    edge_attr=1,
    create_using=networkx.MultiDiGraph())
G
print(f"Edges:{G.number_of_edges()}, Nodes:{G.number_of_nodes()}, Avg Degree:{sum(dict(G.degree()).values())/G.number_of_nodes()}, Density:{networkx.density(G)}")
G

Edges:97246, Nodes:2505, Avg Degree:77.64151696606787, Density:0.015503497796738791


## R approach

From the R package `sna`. Treat as the ground truth for a real krackhardt score.

In [25]:
base = importr("base")
sna = importr("sna")

krackhardt_score = sna.hierarchy(networkx.to_numpy_array(G), measure="krackhardt")
print(sna.hierarchy(networkx.to_numpy_array(networkx.DiGraph(G)), measure="krackhardt"))
print(krackhardt_score)

[0.99998939]
[0.99998939]


Adjacency matrix doesn't change from MultiDi to Di

## Approach A

Based on ChatGPT suggestions. UTTER BOLLOCKS

In [19]:
# Convert the multigraph to a directed graph
G = networkx.DiGraph(G)
print(f"Edges:{G.number_of_edges()}, Nodes:{G.number_of_nodes()}, Avg Degree:{sum(dict(G.degree()).values())/G.number_of_nodes()}, Density:{networkx.density(G)}")

# Compute the in-degree and out-degree centrality for each node
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

# Create a dictionary to store the hierarchy scores for each node
hierarchy_scores = {}

# Compute the hierarchy score for each node
for node in G.nodes():
    # Compute the number of higher-status nodes that the node is connected to
    higher_status_nodes = set()
    for neighbour in G.successors(node):
        if in_degrees[neighbour] > in_degrees[node]:
            higher_status_nodes.add(neighbour)

    # Compute the number of lower-status nodes that the node is connected to
    lower_status_nodes = set()
    for neighbour in G.predecessors(node):
        if out_degrees[neighbour] > out_degrees[node]:
            lower_status_nodes.add(neighbour)

    # Compute the hierarchy score as the difference between the number of higher-status
    # and lower-status nodes that the node is connected to
    hierarchy_scores[node] = len(higher_status_nodes) - len(lower_status_nodes)


Edges:94643, Nodes:2505, Avg Degree:75.56327345309381, Density:0.015088513069707231


In [9]:
sum(hierarchy_scores.values())/len(hierarchy_scores)

2.5636726546906186

## Krackhardt OG approach

From his OG paper:

The graph hierarchy condition states that in a digraph D, for each pair of points where one (Pi) can reach another (Pj), the second (Pj) can't reach the first (Pi). 
For example, in a formal organization chart a high lvl employee can reach through the chain of command her subordinate's subordinate. If the formal organization if working "properly", this lower lvl employee can't simultaneously reach the high lvl employee.
To measure the degree of hierarchy of digraph D, a new digraph Dr must be created. Dr is defined as the reachability digraph of D. Each point in D exists in Dr; moreover, the line (Pi,Pj) exists in Dr if and only if Pi can reach Pj in D. If D is graph hierarchic, then Dr will have no symmetric lines in it (i.e. if the line (Pi,Pj) exists in Dr then the line (Pj,Pi) does not).

The degree of hierarchy then is defined as:

    Graph Hierarchy = 1 - [V/MaxV]
    Where:
        V = Number of unordered pairs of points in Dr that are symmetrically linked
        MaxV = Number of unordered pairs of points in Dr where Pi is linked to Pj or viceversa.

In [41]:
#First, compute reachability graph
diG = networkx.DiGraph(G)

for node in diG.nodes():
    print(node)
    print(networkx.descendants(diG, node))

CXCL1
{'ECE2', 'ARHGEF16', 'OPN5', 'RGS12', 'FGD1', 'PDE11A', 'PDE3A', 'PDE4A', 'CAMK2B', 'PRKAR2B', 'GNG11', 'NLN', 'KALRN', 'GNG12', 'MCF2', 'CXCR2', 'RASGRP2', 'PDE2A', 'RHOA', 'PRKCB', 'PIK3CG', 'RGS20', 'CXCR1', 'TIAM1', 'BTK', 'GPR143', 'ARRB1', 'GNB4', 'ARHGEF2', 'ROCK2', 'DGKD', 'TRPC3', 'ARHGEF7', 'PLCB3', 'PIK3R6', 'ADCY1', 'PLCB4', 'CAMKK1', 'RGS4', 'GNG7', 'PAK1', 'ADCY4', 'KPNA2', 'TAS2R9', 'GNAQ', 'ADCY6', 'OBSCN', 'PPP1CA', 'DGKH', 'ARHGEF10L', 'PLA2G4A', 'ARHGEF3', 'ECE1', 'XK', 'ADCY7', 'PDE8B', 'PIK3CA', 'SOS2', 'PRKCG', 'ARHGEF6', 'ITSN1', 'ABHD12', 'TAS2R50', 'GNA14', 'PRKCA', 'DAGLA', 'PDE3B', 'PLEKHG5', 'TAS1R2', 'TRPC6', 'ARHGEF9', 'PRKCE', 'TAS1R3', 'GNG2', 'RHOB', 'CAMK4', 'PRKACG', 'RASGRF2', 'RGS18', 'GNB3', 'ABR', 'TRPC7', 'RRH', 'CDK5', 'GNB5', 'ADCY9', 'PPP2R5D', 'RGR', 'PDE4B', 'GPHB5', 'PRKCQ', 'PIK3R1', 'VAV1', 'GNG10', 'GNA12', 'DGKZ', 'TAAR2', 'PRKACA', 'GPSM3', 'TAS2R8', 'ROCK1', 'GRK5', 'GPSM2', 'ARHGEF17', 'P2RY11', 'OPN1LW', 'AKT3', 'RGS10', 'RGS2